In [87]:
# permite a utilização de caminhos relativos ao projeto, mesmo no diretório de notebooks
import numpy as np
import random
from knapsax.optimization import Knapsack, Solution
import matplotlib.pyplot as plt
from knapsax.populational import Populational

from knapsax.utils import setrootdir

setrootdir("knapsax")

'Directory knapsax successfully loaded as current working directory.'

In [88]:
knapsack = Knapsack(instance_file="data/knapsack-instance.txt")
knapsack

Knapsack(file=data/knapsack-instance.txt, n_items=100, capacity=1550)

# PSO

In [89]:
def particula(n):
    posicao = [random.randint(0, 1) for _ in range(n)]
    velocidade = [random.uniform(-1, 1) for _ in range(n)]
    return [posicao, posicao.copy(), velocidade]

In [90]:
def avaliar(pos, itens, capacidade):
    peso_total = 0
    valor_total = 0
    for selecionado, item in zip(pos, itens):
        if selecionado:
            peso_total += item.weight
            valor_total += item.value
    if peso_total > capacidade:
        return 0.01
    return valor_total

In [91]:
def atualizar_pbest(p, itens, capacidade):
    if avaliar(p[0], itens, capacidade) > avaliar(p[1], itens, capacidade):
        p[1] = p[0].copy()

In [92]:
def obter_gbest(pop, itens, capacidade):
    return max(pop, key=lambda p: avaliar(p[1], itens, capacidade))[1]

In [93]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [94]:
def atualizar_velocidade(p, gbest, w, c1, c2):
    n = len(p[0])
    r1 = np.random.rand(n)
    r2 = np.random.rand(n)
    pos = np.array(p[0])
    pbest = np.array(p[1])
    gbest = np.array(gbest)
    vel = np.array(p[2])
    nova = w * vel + c1 * r1 * (pbest - pos) + c2 * r2 * (gbest - pos)
    p[2] = nova.tolist()

In [95]:
def atualizar_posicao(p):
    nova_pos = []
    for v in p[2]:
        prob = sigmoid(v)
        nova_pos.append(1 if random.random() < prob else 0)
    p[0] = nova_pos

In [105]:
def pso_mochila(knapsack: Knapsack, num_particulas=30, max_iter=500, w=0.7, c1=1.5, c2=1.5):
    n = knapsack.n_items
    itens = knapsack.items
    capacidade = knapsack.capacity

    pop = [particula(n) for _ in range(num_particulas)]
    historico = []

    for _ in range(max_iter):
        for p in pop:
            atualizar_pbest(p, itens, capacidade)

        gbest = obter_gbest(pop, itens, capacidade)
        historico.append(avaliar(gbest, itens, capacidade))

        for p in pop:
            atualizar_velocidade(p, gbest, w, c1, c2)
            atualizar_posicao(p)

    return gbest, avaliar(gbest, itens, capacidade), historico


In [106]:
solucao, valor_total, historico = pso_mochila(knapsack)

print("Valor total da mochila:", valor_total)
print("Itens selecionados:")
for i, bit in enumerate(solucao):
    if bit:
        item = knapsack.items[i]
        print(f"Item {i}: valor = {item.value}, peso = {item.weight}")


Valor total da mochila: 2069
Itens selecionados:
Item 5: valor = 30, peso = 15
Item 8: valor = 44, peso = 28
Item 13: valor = 55, peso = 40
Item 14: valor = 33, peso = 23
Item 16: valor = 18, peso = 12
Item 18: valor = 42, peso = 31
Item 20: valor = 51, peso = 37
Item 21: valor = 68, peso = 54
Item 22: valor = 39, peso = 25
Item 26: valor = 29, peso = 21
Item 32: valor = 71, peso = 56
Item 33: valor = 54, peso = 39
Item 35: valor = 26, peso = 18
Item 36: valor = 77, peso = 62
Item 37: valor = 63, peso = 47
Item 38: valor = 45, peso = 29
Item 40: valor = 22, peso = 14
Item 41: valor = 69, peso = 53
Item 42: valor = 52, peso = 36
Item 44: valor = 37, peso = 24
Item 45: valor = 61, peso = 49
Item 49: valor = 57, peso = 42
Item 51: valor = 24, peso = 17
Item 53: valor = 53, peso = 38
Item 56: valor = 66, peso = 51
Item 58: valor = 49, peso = 34
Item 59: valor = 74, peso = 61
Item 60: valor = 41, peso = 26
Item 63: valor = 94, peso = 80
Item 64: valor = 19, peso = 13
Item 67: valor = 60, pe

AttributeError: 'list' object has no attribute 'items'